In [4]:
import multiprocessing
import threading
import tensorflow as tf
import numpy as np
import gym
import os
import shutil
import matplotlib.pyplot as plt

Policy neural network  

Implements network which takes in input and produces actions  
and log probabilities given a sampling distribution parameterization.

In [5]:
GAME = 'Pendulum-v0'
env = gym.make(GAME)

class spaces(object):
    discrete = 0
    box = 1

def get_space(space):
    if hasattr(space, 'n'):
        return space.n, spaces.discrete, None
    elif hasattr(space, 'shape'):
        return np.prod(space.shape), spaces.box, (space.low, space.high)

def get_spaces(spaces):
    if hasattr(spaces, 'spaces'):
        return zip(*[get_space(space) for space in spaces.spaces])
    else:
        return [(ret,) for ret in get_space(spaces)]

def sampling_dim(dim, typ):
    if typ == spaces.discrete:
        return dim
    elif typ == spaces.box:
        return 2 * dim  # Gaussian mean and std
    else:
        assert False
        
obs_space = env.observation_space
obs_dims, obs_types, obs_info = get_spaces(obs_space)
act_space = env.action_space
act_dims, act_types, act_info = get_spaces(act_space)
total_obs_dim = sum(obs_dims)
total_sampled_act_dim = sum(act_dims)

obs_dims_and_types = zip(obs_dims, obs_types)
act_dims_and_types = zip(act_dims, act_types)

Evaluate where log_probs com from and network evaluation is performed.  
Starting point in model tensorflow setup:

In [ ]:
# Get log_probs
(self.policy_internal_states, self.logits, self.log_probs, self.entropies, self.self_kls) = \
                    self.policy.multi_step(self.observations,
                                           self.internal_state,
                                           self.actions)

Two different kinds of baseline are availabe:
* Recurrent policy
* MLPPolicy --> Non-recurrent policy --> inherits from Recurrent Policy

Input of neural network is dependend of if prev. action is also considered as input:
* If not only dim_obs
* If yes dim_obs + dim_actions

In [ ]:
def input_dim(self):
    return (self.env_spec.total_obs_dim + 
            self.env_spec.total_sampled_act_dim * self.input_prev_actions)

Output dimension is in the continouse case 2 * act_dim to model Gaussian means and sd

In [ ]:
def output_dim(self):
    return self.env_spec.total_sampling_act_dim

The fucntion core(obs, prev_internal_sate, prev_actions) implements the core neural network:
* taking in inputs 
* outputting sampling distribution parameters

At first get Batch Size

In [ ]:
batch_size = tf.shape(obs[0])[0]

Next check if recurrent network is applied - If not initialize prev_internal_state with:
* [batch_size, rnn_state_dim] of zeros
* Otherwise for every Episode in the Batch one rnn_state_dim is available

In [ ]:
if not self.recurrent:
      prev_internal_state = tf.zeros([batch_size, self.rnn_state_dim])

Next an RNN-Cell is created via "get_cell()"

In [ ]:
cell = self.get_cell()

The cells input dim is half the dimension of the internal (rnn) dim 

In [ ]:
self.cell_input_dim = self.internal_dim // 2

Afterwards an tf.contrib.rnn.LSTMCell is created
* state_ist_tuple: If True, accepted and returned states are 2-tuples of the c_state and m_state
* (optional) Python boolean describing whether to reuse variables in an existing scope

In [ ]:
cell = tf.contrib.rnn.LSTMCell(self.cell_input_dim,
                               state_is_tuple=False,
                                reuse=tf.get_variable_scope().reuse)

Rnn models return:
* outputs [batch_size, max_time, output_size] --> actually desired output
* The final state [batch_size] + cell.state_size

The LSTM Cell is wrapped into an "OutputProjectionWrapper"
* Maps from cell_input_dim to output dim

In [ ]:
cell = tf.contrib.rnn.OutputProjectionWrapper(
            cell,
            self.output_dim,
            reuse=tf.get_variable_scope().reuse)

Now the One-Layer Lstm Cell with Output Layer is created and we are back at the "core" function
* go on now with the weight creation

In [ ]:
b = tf.get_variable('input_bias', [self.cell_input_dim], initializer=self.vector_init)
cell_input = tf.nn.bias_add(tf.zeros([batch_size, self.cell_input_dim]), b)

Iterate now through all observation kinds and get type (discrete or box) and dimension

In [ ]:
w = tf.get_variable('w_state%d' % i, [obs_dim, self.cell_input_dim], initializer=self.matrix_init)

In case of box - like pendulum

In [ ]:
 cell_input += tf.matmul(obs[i], w)

Im Fall, dass die vorhergehende Action ebenfalls inkludiert werden soll

In [ ]:
w = tf.get_variable('w_prev_action%d' % i, [act_dim, self.cell_input_dim], initializer=self.matrix_init)
cell_input += tf.matmul(prev_actions[i], w)

Anschließend wird der Cell Input in die LSTM Zelle gefuettert sowie der vorhergehende internal state, sollte die Episode noch nicht abgeschlossen wurden sein:
* Output isr von der Form [batch_size, rnn_state_dim] also für jede episode ein interne dimension
* Output [batch_size, max_len, total_sampling_action_dim] --> Im Fall vom Pendulum (mu, sigma)

In [ ]:
output, next_state = cell(cell_input, prev_internal_state)


With this functionallity availabe one can compute actions and corresponding log_probs with the following Hierarchy:
* multi_step --> Calculate log-probs and other calculations on batch of episodes
* single_step --> Single RNN step.  Equivalently, single-time-step sampled actions.
* sample_actions --> Sample all actions given output of core network
* sample_action --> Sample an action from a distribution.

Lets start with sample_actions

In [ ]:
act_logits = output[:, start_idx:start_idx + act_dim] # Get Mu, Sigma

In [ ]:
act = self.sample_action(act_logits,
                         sampling_dim,
                         act_dim,
                         act_type,
                         greedy=greedy)

In [ ]:
means = logits[:, :sampling_dim / 2]
std = logits[:, sampling_dim / 2:]
# If greedy 
act = means
# Else
act = means + std * tf.random_normal([batch_size, act_dim])

Finallys "log_prob_action" based on the output of the NN, the choosen  act

In [ ]:
act_log_prob = self.log_prob_action(act, act_logits, sampling_dim, act_dim, act_type)

In [ ]:
means = logits[:, :sampling_dim / 2]
std = logits[:, sampling_dim / 2:]
# Log normal distribution
act_log_prob = (- 0.5 * tf.log(2 * np.pi * tf.square(std))- 0.5 * tf.square(action - means) / tf.square(std))
# If action is means
# act_log_prob = - 0.5 * tf.log(2 * np.pi * tf.square(std))
act_log_prob = tf.reduce_sum(act_log_prob, -1) --> In case multidimensional action got applied

The sampled action, the log probs, the logits (Output of the neural net) as well as entropy and self_kl (KL-Divergence) are returned

Those outputs and the next internal state of the RNN-Network are collected by the "single_step" function

The multi_step function "Calculate log-probs and other calculations on batch of episodes." and calls "single_step" for all observation and action pairs via tf.scan

In [ ]:
(internal_states, _, logits, log_probs, entropies, self_kls) = tf.scan(
                            self.single_step,
                            (all_obs, all_actions),
                            initializer=self.get_initializer(
                            batch_size, initial_state, initial_actions))

* Therefore log_probs are calculated based on the output of the NN (logits)
* If one considers the unified version of the PCL the  values are also calculated by the logits

In [21]:
def f_transform(q_values, tau):
    max_q = tf.reduce_max(q_values, -1, keep_dims=True)
    return tf.squeeze(max_q, [-1]) + tau * tf.log(
                    tf.reduce_sum(tf.exp((q_values - max_q) / tau), -1))

Via the following call:

In [ ]:
values = f_transform((self.tau + self.eps_lambda) * reshaped_policy_logits[0], (self.tau + self.eps_lambda))

Input to the function $$(\tau + \lambda) * Output_{NN}$$  
If we consider der unified PCL version the values can be computed in the following way:
$$V_p(s) =  \tau \sum_a exp \{ \frac{Q_p(s,a)}{\tau}\}$$